# **House price prediction of houses in Pune,India**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("E:\BharatIntern\Pune_house_price_prediction_model\Pune house data.csv")

<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Dell\AppData\Local\Temp\ipykernel_6160\801920349.py:1: SyntaxWarning: invalid escape sequence '\B'
  df=pd.read_csv("E:\BharatIntern\Pune_house_price_prediction_model\Pune house data.csv")


In [3]:
# accessing top 5 rows from our dataset
df.head()

,availability,beds,baths,balcony,total_sqft,area_type,price,society,site_location
0,19-Dec,2 BHK,2.0,1.0,1056,Super built-up Area,39.07,Coomee,Alandi Road
1,Ready To Move,4 Bedroom,5.0,3.0,2600,Plot Area,120.00,Theanmp,Ambegaon Budruk
2,Ready To Move,3 BHK,2.0,3.0,1440,Built-up Area,62.00,NaN,Anandnagar
3,Ready To Move,3 BHK,3.0,1.0,1521,Super built-up Area,95.00,Soiewre,Aundh
4,Ready To Move,2 BHK,2.0,1.0,1200,Super built-up Area,51.00,NaN,Aundh Road


In [4]:
# As in prediction of price we won't be requiring data such as availability,site_location and society so dropping columns.
df.drop(["availability","society","site_location"],axis="columns",inplace=True)

In [5]:
#concise summary of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   beds        13304 non-null  object 
 1   baths       13247 non-null  float64
 2   balcony     12711 non-null  float64
 3   total_sqft  13320 non-null  object 
 4   area_type   13320 non-null  object 
 5   price       13320 non-null  float64
dtypes: float64(3), object(3)
memory usage: 624.5+ KB


In [6]:
#descriptive stats of the dataframe
df.describe()

,baths,balcony,price
count,13247.000000,12711.000000,13320.000000
mean,2.692610,1.584376,112.565627
std,1.341458,0.817263,148.971674
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [7]:
#checking count of null values
df.isnull().sum()

beds           16
baths          73
balcony       609
total_sqft      0
area_type       0
price           0
dtype: int64

In [8]:
# viewing the shape of the dataframe
df.shape

(13320, 6)

In [9]:
#making copy the dataset
dfCopy = df.copy()

In [10]:
#ASSUMPTION : how record containing balcony as null value do not have balcony
dfCopy['balcony'].fillna(value=0.0,inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_6160\3246045921.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfCopy['balcony'].fillna(value=0.0,inplace=True)


In [11]:
dfCopy.isnull().sum()

beds          16
baths         73
balcony        0
total_sqft     0
area_type      0
price          0
dtype: int64

In [12]:
# dropping records having size null
dfCopy.dropna(subset=['beds'], inplace=True)

In [13]:
dfCopy.isnull().sum()

beds           0
baths         57
balcony        0
total_sqft     0
area_type      0
price          0
dtype: int64

In [14]:
dfCopy['beds'] = dfCopy['beds'].astype(str)
dfCopy['beds'] = dfCopy['beds'].apply(lambda x : (x.split(' ')[0]))

In [15]:
dfCopy = dfCopy[dfCopy['beds'] != 'nan']

In [16]:
dfCopy.head()

,beds,baths,balcony,total_sqft,area_type,price
0,2,2.0,1.0,1056,Super built-up Area,39.07
1,4,5.0,3.0,2600,Plot Area,120.00
2,3,2.0,3.0,1440,Built-up Area,62.00
3,3,3.0,1.0,1521,Super built-up Area,95.00
4,2,2.0,1.0,1200,Super built-up Area,51.00


In [17]:
dfCopy.isna().sum()

beds           0
baths         57
balcony        0
total_sqft     0
area_type      0
price          0
dtype: int64

In [18]:
dfcopy1 = dfCopy.groupby('beds')['baths'].agg('count').reset_index(name='baths_count')
dfcopy2 = dfCopy.groupby('beds')['beds'].agg('count').reset_index(name='beds_count')

# Merge the DataFrames on the 'beds' column
result = pd.merge(dfcopy1, dfcopy2, on='beds')

In [19]:
result

,beds,baths_count,beds_count
0,1,649,656
1,10,14,14
2,11,4,4
3,12,1,1
4,13,1,1
5,14,1,1
6,16,1,1
7,18,1,1
8,19,1,1
9,2,5527,5528


In [20]:
result['bath_to_beds_ratio'] = result.apply(
    lambda row: (row['baths_count'] / row['beds_count']) if row['beds_count'] != 0 else None, axis=1)


In [21]:
result

,beds,baths_count,beds_count,bath_to_beds_ratio
0,1,649,656,0.989329
1,10,14,14,1.000000
2,11,4,4,1.000000
3,12,1,1,1.000000
4,13,1,1,1.000000
5,14,1,1,1.000000
6,16,1,1,1.000000
7,18,1,1,1.000000
8,19,1,1,1.000000
9,2,5527,5528,0.999819


In [22]:
# As we can see almost all bath_to_size_ratio is close to 1 then we can assume the null values to be 1 in bath column
dfCopy['baths'].fillna(value=1, inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_6160\4199639419.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfCopy['baths'].fillna(value=1, inplace=True)


In [23]:
dfCopy.isnull().sum()

beds          0
baths         0
balcony       0
total_sqft    0
area_type     0
price         0
dtype: int64

In [24]:
dfCopy

,beds,baths,balcony,total_sqft,area_type,price
0,2,2.0,1.0,1056,Super built-up Area,39.07
1,4,5.0,3.0,2600,Plot Area,120.00
2,3,2.0,3.0,1440,Built-up Area,62.00
3,3,3.0,1.0,1521,Super built-up Area,95.00
4,2,2.0,1.0,1200,Super built-up Area,51.00
...,...,...,...,...,...,...
13315,5,4.0,0.0,3453,Built-up Area,231.00
13316,4,5.0,0.0,3600,Super built-up Area,400.00
13317,2,2.0,1.0,1141,Built-up Area,60.00
13318,4,4.0,1.0,4689,Super built-up Area,488.00


In [25]:
dfCopy['beds'] = dfCopy['beds'].astype(float)

In [26]:
# Converting the range values to integer values and removing other types of error
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

dfCopy['size'] = dfCopy.total_sqft.apply(convert_sqft_to_num)
dfCopy = dfCopy.drop('total_sqft', axis='columns')
dfCopy.head()

,beds,baths,balcony,area_type,price,size
0,2.0,2.0,1.0,Super built-up Area,39.07,1056.0
1,4.0,5.0,3.0,Plot Area,120.00,2600.0
2,3.0,2.0,3.0,Built-up Area,62.00,1440.0
3,3.0,3.0,1.0,Super built-up Area,95.00,1521.0
4,2.0,2.0,1.0,Super built-up Area,51.00,1200.0


In [27]:
dfCopy['baths'] = dfCopy['baths'].astype(float)
dfCopy['balcony'] = dfCopy['balcony'].astype(float)
dfCopy['price'] = dfCopy['price'].astype(float)

In [28]:
dfCopy.isnull().sum()

beds          0
baths         0
balcony       0
area_type     0
price         0
size         46
dtype: int64

In [29]:
dfCopy = dfCopy.dropna(subset=["size"])

In [30]:
dfCopy.isnull().sum()

beds         0
baths        0
balcony      0
area_type    0
price        0
size         0
dtype: int64

In [31]:
dfCopy.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2402
Carpet  Area              86
Plot  Area              1991
Super built-up  Area    8779
Name: area_type, dtype: int64

In [32]:
print(dfCopy['area_type'].unique())


['Super built-up  Area' 'Plot  Area' 'Built-up  Area' 'Carpet  Area']


In [33]:
dfCopy['area_type'] = dfCopy['area_type'].str.strip().str.title()


In [34]:
print(dfCopy['area_type'].unique())

['Super Built-Up  Area' 'Plot  Area' 'Built-Up  Area' 'Carpet  Area']


In [35]:
# replacing with integer values in area_type
dfCopy['area_type'] = dfCopy['area_type'].replace({
    'Super Built-Up  Area': 1,
    'Built-Up  Area': 2,
    'Plot  Area': 3,
    'Carpet  Area': 4
})


C:\Users\Dell\AppData\Local\Temp\ipykernel_6160\724535902.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfCopy['area_type'] = dfCopy['area_type'].replace({


In [36]:
pd.set_option('future.no_silent_downcasting', True)
dfCopy.head()

,beds,baths,balcony,area_type,price,size
0,2.0,2.0,1.0,1,39.07,1056.0
1,4.0,5.0,3.0,3,120.00,2600.0
2,3.0,2.0,3.0,2,62.00,1440.0
3,3.0,3.0,1.0,1,95.00,1521.0
4,2.0,2.0,1.0,1,51.00,1200.0


In [37]:
dfCopy['area_type'] = dfCopy['area_type'].astype(int)

In [38]:
new_order = ['beds', 'baths', 'balcony', 'area_type', 'size', 'price']
dfCopy = dfCopy[new_order]

In [39]:
dfCopy.head()

,beds,baths,balcony,area_type,size,price
0,2.0,2.0,1.0,1,1056.0,39.07
1,4.0,5.0,3.0,3,2600.0,120.00
2,3.0,2.0,3.0,2,1440.0,62.00
3,3.0,3.0,1.0,1,1521.0,95.00
4,2.0,2.0,1.0,1,1200.0,51.00


In [40]:
# saving our final dataset
dfCopy.to_csv('final_pune_house_data.csv')

In [41]:
# defining the feature and the target
x = dfCopy.drop(['price'],axis=1)
y = dfCopy['price']

In [42]:
# splitting dataset into training and testing data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [43]:
# Creating and training Linear regression model
from sklearn.linear_model import LinearRegression


#defining the model
LinearReg = LinearRegression()

#training the model
LinearReg.fit(x_train, y_train)

LinearRegression()

In [44]:
# testing the model
predictions = LinearReg.predict(x_test)

In [45]:
# evaluating the model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

LinearRegression_MAE = mean_absolute_error(y_test, predictions)
LinearRegression_MSE = mean_squared_error(y_test, predictions)
LinearRegression_R2 = r2_score(y_test, predictions)

In [46]:
# applying KNN algorithm
from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors=4)
KNN.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=4)

In [47]:
predictions = KNN.predict(x_test)

In [48]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

KNN_mean_absolute_error = mean_absolute_error(y_test, predictions)
KNN_mean_squared_error = mean_squared_error(y_test, predictions)
KNN_r2_Score = r2_score(y_test, predictions)

In [49]:
# applying decision tree algorithm
from sklearn.tree import DecisionTreeRegressor

Tree = DecisionTreeRegressor(random_state=10)
Tree.fit(x_train, y_train)

DecisionTreeRegressor(random_state=10)

In [50]:
predictions = Tree.predict(x_test)

In [51]:
Tree_mean_absolute_error = mean_absolute_error(y_test, predictions)
Tree_mean_squared_error = mean_squared_error(y_test, predictions)
Tree_r2_Score = r2_score(y_test, predictions)

In [52]:
# applying random forest
from sklearn.ensemble import RandomForestRegressor

RF = RandomForestRegressor(n_estimators=100, random_state=42)
RF.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [53]:
predictions = RF.predict(x_test)
predictions

array([ 42.34044083,  91.62586   , 130.16399483, ...,  97.12902381,
        44.45106548, 267.02333333])

In [54]:
RF_mean_absolute_error = mean_absolute_error(y_test, predictions)
RF_mean_squared_error = mean_squared_error(y_test, predictions)
RF_r2_Score = r2_score(y_test, predictions)

In [55]:
Report = pd.DataFrame({'MAE': [LinearRegression_MAE, KNN_mean_absolute_error, Tree_mean_absolute_error, RF_mean_absolute_error],
                       'MSE': [LinearRegression_MSE, KNN_mean_squared_error, Tree_mean_squared_error, RF_mean_squared_error],
                       'r2-Score': [LinearRegression_R2, KNN_r2_Score, Tree_r2_Score, RF_r2_Score]},
                     index=['Linear Regression','KNN', 'Decision Tree', 'Random Forest'])

Report

,MAE,MSE,r2-Score
Linear Regression,43.181427,10656.239474,0.501623
KNN,38.391059,10680.152528,0.500504
Decision Tree,42.573796,13541.850613,0.366667
Random Forest,35.825593,7763.012679,0.636935


In [56]:
from sklearn.linear_model import Ridge
ridg = Ridge()
ridg.fit(x_train, y_train)

Ridge()

In [57]:
predictions = RF.predict(x_test)
predictions

array([ 42.34044083,  91.62586   , 130.16399483, ...,  97.12902381,
        44.45106548, 267.02333333])

In [58]:
ridg_mean_absolute_error = mean_absolute_error(y_test, predictions)
ridg_mean_squared_error = mean_squared_error(y_test, predictions)
ridg_r2_Score = r2_score(y_test, predictions)

In [59]:
ridg_r2_Score

0.6369348306608171

In [60]:
# now saving the random forest model as it has highest r2_score and lowest errors
import pickle

pickle.dump(RF, open('RFmodel.pkl','wb'))